In [ ]:
import os
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    AutoModelForSeq2SeqLM
)
from langchain import (
    LLMChain,
    PromptTemplate
)
from langchain.llms import (
    OpenAI,
    HuggingFacePipeline,
    HuggingFaceHub
)
from langchain.chains import ConversationChain
from langchain.agents import (
    load_tools,
    initialize_agent,
    AgentType
)

# Init LLMs

### OpenAI API

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
llm = OpenAI(temperature=0)

### Huggingface API

In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''
# model_name = "google/flan-t5-xxl"
model_name = "bigscience/bloom-560m"
llm = HuggingFaceHub(
    repo_id=model_name,
    task="text-generation",
    model_kwargs={'temperature':1e-10}
)

### Local Model

In [ ]:
# model_name = "/Users/zeyesun/Documents/Data/models/bloomz-560m"
model_name = "D:\\Data\\models\\bloomz-560m"
max_length_generation = 512
local_rank = 0
do_sample = False
num_return_sequences = 1
top_k = 10
top_p = 0.9
temperature = 0.9
repetition_penalty = 1.0

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map={"": local_rank},
    max_new_tokens=max_length_generation,
#     eos_token_id=tokenizer.bos_token_id,
    do_sample=do_sample,
    num_return_sequences=num_return_sequences,
    top_k=top_k,
    top_p=top_p,
    temperature=temperature,
    repetition_penalty=repetition_penalty
)
llm = HuggingFacePipeline(pipeline=pipe)

# Applications

### 1. QA

In [ ]:
template = """Question: {question} Answer: """
# template = """[Round 1]\n\n问：{question}\n\n答："""

In [ ]:
prompt = PromptTemplate(template=template, input_variables=['question'])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
questions = [
    "What is the capital of France?",
    "What is the capital of England?",
    "Which NFL team won the Super Bowl in the 2010 season?",
    "你是谁"
]
for question in questions:
#     print(llm(prompt.format(question=question)))
    print(llm_chain.run(question))

### 2. Conversation

In [ ]:
conversation = ConversationChain(llm=llm, verbose=True)
conversation.predict(input="Hi there!")
conversation.predict(input="Can we talk about AI?")
conversation.predict(input="I'm interested in Reinforcement Learning.")

### 3. Wikipedia Search

In [ ]:
tools = load_tools(["wikipedia"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("In what year was the film Departed with Leopnardo Dicaprio released?")

In [ ]:
tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")

### 4. Google Search

In [ ]:
os.environ["SERPAPI_API_KEY"] = ''
tool_names = ["serpapi"]
tools = load_tools(tool_names)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("What's the date today? What great events have taken place today in history?")